In [2]:
import os
%pwd

'c:\\PROJECTS\\Credit-Risk-Classification\\research'

In [3]:
os.chdir("../") ## Helps to go back to the directory
%pwd

'c:\\PROJECTS\\Credit-Risk-Classification'

In [4]:
## Entity code
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from CreditRisk.constants import *
from CreditRisk.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
## Components
import os
import urllib.request as request
import zipfile
from CreditRisk import logger
from CreditRisk.utils.common import get_size

In [8]:
class DataIngestion:
    """
    Initialize the DataIngestion instance.

    Args:
        config (DataIngestionConfig): Configuration for data ingestion.
    """
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        """
    Download the file from the specified URL and save it locally.

    Returns:
        None
    """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
## Make sure to add some dummy values (key: val) in the yaml files to avoid error


try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-10 17:24:27,658: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-10 17:24:27,659: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-10 17:24:27,660: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-10 17:24:27,661: INFO: common: created directory at: artifacts]
[2023-08-10 17:24:27,663: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-10 17:24:28,460: INFO: 1623501443: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 10258
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3f3f328db076c07f8a22a25c84f420ce4973663fc8ac1aa9e0763700bd1e1270"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1644:8269:25B300:2D71FF:64D4CFE9
Accept-Ranges: bytes
Date: Thu, 10 A